## VectorStoreRetriever란?
- VectorStore를 Retriever 인터페이스로 변환하는 가벼운 래퍼 클래스
- Vector Store의 검색 메서드(similarity search, MMR 등)를 사용하여 문서를 검색
- Runnable 인터페이스를 구현하여 LCEL 체인에 쉽게 통합 가능

### 핵심 개념
- **VectorStore**: 벡터 유사도를 기반으로 문서를 저장하고 검색하는 데이터베이스
- **Retriever**: 문서 검색을 위한 표준 인터페이스를 제공하는 추상화
- **LCEL 체인**: VectorStoreRetriever는 Runnable이므로 다른 구성요소와 체인으로 연결 가능

### VectorStoreRetriever의 장점
- **표준화된 인터페이스**: 다양한 Vector Store를 동일한 방식으로 사용 가능
- **체인 통합**: `invoke()`, `batch()` 등의 표준 메서드 지원
- **유연한 검색**: 다양한 검색 유형과 파라미터 지원


실습할 문서 만들기

In [ ]:
from langchain_core.documents import Document

# 군용 무기 관련 샘플 문서 생성
documents = [
    Document(
        page_content="K2 흑표는 대한민국이 개발한 3.5세대 주력전차로, 120mm 주포와 복합장갑을 갖추고 있습니다.",
        metadata={"source": "국방백서", "category": "기갑차량", "년도": "2021"}
    ),
    Document(
        page_content="K9 자주포는 155mm 구경의 자주포로, 최대 사거리 40km를 자랑하는 한국형 자주포입니다.",
        metadata={"source": "방산업체", "category": "포병장비", "년도": "2020"}
    ),
    Document(
        page_content="KAI KF-21 보라매는 한국형 전투기 사업으로 개발된 4.5세대 다목적 전투기입니다.",
        metadata={"source": "항공우주", "category": "항공기", "년도": "2022"}
    ),
    Document(
        page_content="천궁 방공미사일은 중거리 지대공 미사일로 탄도미사일 요격 능력을 보유하고 있습니다.",
        metadata={"source": "방위산업", "category": "미사일", "년도": "2021"}
    ),
    Document(
        page_content="독도함은 대형수송함으로 상륙작전 지원과 헬기 운용이 가능한 다목적 함정입니다.",
        metadata={"source": "해군본부", "category": "함정", "년도": "2020"}
    ),
    Document(
        page_content="수리온 헬기는 한국형 기동헬기로 다양한 임무에 활용되는 다목적 헬리콥터입니다.",
        metadata={"source": "육군본부", "category": "헬리콥터", "년도": "2019"}
    ),
    Document(
        page_content="현무 미사일은 지대지 전술미사일로 정밀타격 능력을 갖춘 한국형 미사일입니다.",
        metadata={"source": "국방과학연구소", "category": "미사일", "년도": "2021"}
    ),
    Document(
        page_content="세종대왕급 구축함은 이지스 시스템을 탑재한 대한민국 해군의 최신 구축함입니다.",
        metadata={"source": "해군본부", "category": "함정", "년도": "2020"}
    )
]

print(f"준비된 군용 무기 문서 수: {len(documents)}")
print(f"첫 번째 문서: {documents[0].page_content}")
print(f"문서 카테고리들: {set([doc.metadata['category'] for doc in documents])}")


임베딩 객체 생성

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 한국어 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1",  # 한국어 임베딩 모델
    model_kwargs={'device': 'cpu'},  # CPU 사용
    encode_kwargs={'normalize_embeddings': True}  # 벡터 정규화
)

print(f"임베딩 모델: {embeddings.model_name}")


Vector Store 생성

In [ ]:
from langchain_community.vectorstores import FAISS

# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(
    documents=documents, 
    embedding=embeddings
)

print("VectorStore가 성공적으로 생성되었습니다.")


VectorStoreRetriever 생성

In [ ]:
# VectorStore를 Retriever로 변환
retriever = vectorstore.as_retriever()

print(f"Retriever 타입: {type(retriever)}")
print(f"Retriever는 Runnable인가?: {hasattr(retriever, 'invoke')}")


기본 검색 테스트

In [ ]:
# 단일 쿼리 검색
question = "전차에 대해 알려줘"
results = retriever.invoke(question)

print(f"검색 결과 수: {len(results)}")
print("\n=== 검색 결과 ===")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}")
    print(f"   메타데이터: {doc.metadata}")
    print()


### 검색 유형별 설정

**1. 유사도 검색 (기본값)**

In [ ]:
# 유사도 검색 (기본값)
similarity_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}  # 상위 2개 결과만 반환
)

results = similarity_retriever.invoke("미사일 무기")
print("=== 유사도 검색 (k=2) ===")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}")


**MMR (Maximum Marginal Relevance) 검색**  
- 검색된 문서의 다양성과 관련성 간의 균형을 맞추기 위해 사용하는 검색 전략  
- 특히 유사한 결과가 중복되는 것을 방지하고, **사용자의 쿼리와 관련 있으면서도 서로 다른 정보를 제공**받고 싶을 때 사용합니다.

| 매개변수 이름              | 설명                                                   |
| -------------------- | ---------------------------------------------------- |
| `search_type`        | `"mmr"`로 설정하면 MMR 검색 방식 사용. (기본은 `"similarity"`)     |
| `k`                  | 최종적으로 반환할 문서 수 (예: 5이면 5개 문서 반환)                     |
| `fetch_k`            | 후보 문서 수. `k`개를 고르기 위해 먼저 몇 개를 가져올지 (기본값은 보통 20\~100) |
| `lambda_mult`        | λ (람다) 값. 관련성과 다양성 사이의 균형 (0\~1 사이의 값)               |
| `relevance_score_fn` | 유사도를 계산하는 함수. 기본은 cosine similarity, 필요하면 사용자 정의 가능  |
| `filter`             | 검색 전 문서 필터링 조건 (예: 메타데이터 필터링)                        |


In [ ]:
# MMR 검색 - 유사도와 다양성을 모두 고려
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 3,
        "lambda_mult": 0.5  # 0에 가까울수록 다양성 중시, 1에 가까울수록 유사도 중시
    }
)

results = mmr_retriever.invoke("한국 무기")
print("=== MMR 검색 ===")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}")
    print(f"   카테고리: {doc.metadata['category']}")


**6-3. 유사도 점수 임계값 검색**

0.1일때랑 0.35일때 비교해보기

In [ ]:
# 유사도 점수 임계값 검색
threshold_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.35}  # 0.35 이상의 유사도를 가진 문서만 반환
)

results = threshold_retriever.invoke("미사일")
print("=== 유사도 임계값 검색 (threshold=0.35) ===")
print(f"검색된 문서 수: {len(results)}")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}")
